In [1]:
# 会在代码同一级的目录下载训练数据，名称datasets，64M
# n_epochs = 10运行时间约5分钟，为了减少时间改为n_epochs = 2
import torch
from torch.utils.data import DataLoader
from torch.utils.data import random_split
import torch.optim as optim
import numpy as np
from torchvision.transforms import ToTensor
from torchvision.datasets.mnist import MNIST

transform = ToTensor()
train_set = MNIST(root='./datasets', train=True, download=True, transform=transform)
test_set = MNIST(root='./datasets', train=False, download=True, transform=transform)
train_dataloader = DataLoader(train_set, shuffle=True, batch_size=128)
test_dataloader = DataLoader(test_set, shuffle=False, batch_size=128)

# dataset
print("Examples in training set: {}".format(len(train_dataloader.dataset)))
print("Examples in test set: {}".format(len(test_dataloader.dataset)))


Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████████████████████████████████████████████████████████████████████████| 9.91M/9.91M [00:12<00:00, 774kB/s]


Extracting ./datasets\MNIST\raw\train-images-idx3-ubyte.gz to ./datasets\MNIST\raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████████████████████████████████████████████████████████████████████████| 28.9k/28.9k [00:00<00:00, 139kB/s]


Extracting ./datasets\MNIST\raw\train-labels-idx1-ubyte.gz to ./datasets\MNIST\raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████████████████████████████████████████████████████████████████████████| 1.65M/1.65M [00:08<00:00, 201kB/s]


Extracting ./datasets\MNIST\raw\t10k-images-idx3-ubyte.gz to ./datasets\MNIST\raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|█████████████████████████████████████████████████████████████████████████████████████| 4.54k/4.54k [00:00<?, ?B/s]

Extracting ./datasets\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./datasets\MNIST\raw

Examples in training set: 60000
Examples in test set: 10000


In [2]:
from torch import nn
from einops.layers.torch import Rearrange
from torch import Tensor

class PatchEmbedding(nn.Module):
    def __init__(self, in_channels = 1, patch_size = 4, emb_size = 8):
        self.patch_size = patch_size
        super().__init__()
        self.projection = nn.Sequential(
            # break-down the image in s1 x s2 patches and flat them
            Rearrange('b c (h p1) (w p2) -> b (h w) (p1 p2 c)', p1=patch_size, p2=patch_size),
            nn.Linear(patch_size * patch_size * in_channels, emb_size)
        )

    def forward(self, x: Tensor) -> Tensor:
        x = self.projection(x)
        return x

# Run a quick test
sample_datapoint = torch.unsqueeze(train_set[0][0], 0)
print("Initial shape: ", sample_datapoint.shape)
embedding = PatchEmbedding()(sample_datapoint)
print("Patches shape: ", embedding.shape)

Initial shape:  torch.Size([1, 1, 28, 28])
Patches shape:  torch.Size([1, 49, 8])


In [3]:

from einops import rearrange

class Attention(nn.Module):
    def __init__(self, dim, n_heads, dropout):
        super().__init__()
        self.n_heads = n_heads
        self.att = torch.nn.MultiheadAttention(embed_dim=dim,
                                               num_heads=n_heads,
                                               dropout=dropout,
                                               batch_first=True)
        #self.q = torch.nn.Linear(dim, dim)
        #self.k = torch.nn.Linear(dim, dim)
        #self.v = torch.nn.Linear(dim, dim)

    def forward(self, x):
        #q = self.q(x)
        #k = self.k(x)
        #v = self.v(x)
        attn_output, attn_output_weights = self.att(x, x, x)
        return attn_output


In [4]:
'''
class Attention(nn.Module):
    def __init__(self, dim, n_heads, dropout):
        super().__init__()
        self.d = dim
        self.n_heads = n_heads

        assert self.d % n_heads == 0, f"Can't divide dimension {d} into {n_heads} heads"

        d_head = int(self.d / n_heads)
        self.q_mappings = nn.ModuleList([nn.Linear(d_head, d_head) for _ in range(self.n_heads)])
        self.k_mappings = nn.ModuleList([nn.Linear(d_head, d_head) for _ in range(self.n_heads)])
        self.v_mappings = nn.ModuleList([nn.Linear(d_head, d_head) for _ in range(self.n_heads)])
        self.d_head = d_head
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, sequences):
        # Sequences has shape (N, seq_length, token_dim)
        # We go into shape    (N, seq_length, n_heads, token_dim / n_heads)
        # And come back to    (N, seq_length, item_dim)  (through concatenation)
        result = []
        for sequence in sequences:
            seq_result = []
            for head in range(self.n_heads):
                q_mapping = self.q_mappings[head]
                k_mapping = self.k_mappings[head]
                v_mapping = self.v_mappings[head]

                seq = sequence[:, head * self.d_head: (head + 1) * self.d_head]
                q, k, v = q_mapping(seq), k_mapping(seq), v_mapping(seq)

                attention = self.softmax(q @ k.T / (self.d_head ** 0.5))
                seq_result.append(attention @ v)
            result.append(torch.hstack(seq_result))
        return torch.cat([torch.unsqueeze(r, dim=0) for r in result])
'''

'\nclass Attention(nn.Module):\n    def __init__(self, dim, n_heads, dropout):\n        super().__init__()\n        self.d = dim\n        self.n_heads = n_heads\n\n        assert self.d % n_heads == 0, f"Can\'t divide dimension {d} into {n_heads} heads"\n\n        d_head = int(self.d / n_heads)\n        self.q_mappings = nn.ModuleList([nn.Linear(d_head, d_head) for _ in range(self.n_heads)])\n        self.k_mappings = nn.ModuleList([nn.Linear(d_head, d_head) for _ in range(self.n_heads)])\n        self.v_mappings = nn.ModuleList([nn.Linear(d_head, d_head) for _ in range(self.n_heads)])\n        self.d_head = d_head\n        self.softmax = nn.Softmax(dim=-1)\n\n    def forward(self, sequences):\n        # Sequences has shape (N, seq_length, token_dim)\n        # We go into shape    (N, seq_length, n_heads, token_dim / n_heads)\n        # And come back to    (N, seq_length, item_dim)  (through concatenation)\n        result = []\n        for sequence in sequences:\n            seq_result

In [5]:
Attention(dim=8, n_heads=2, dropout=0.1)(torch.ones((1, 49, 8))).shape

torch.Size([1, 49, 8])

In [6]:
class PreNorm(nn.Module):
    def __init__(self, dim, fn):
        super().__init__()
        self.norm = nn.LayerNorm(dim)
        self.fn = fn
    def forward(self, x, **kwargs):
        return self.fn(self.norm(x), **kwargs)

In [7]:
class FeedForward(nn.Sequential):
    def __init__(self, dim, hidden_dim, dropout = 0.):
        super().__init__(
            nn.Linear(dim, hidden_dim),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, dim),
            nn.Dropout(dropout)
        )
ff = FeedForward(dim=32, hidden_dim=32)
ff(torch.ones((1, 1297, 32))).shape

torch.Size([1, 1297, 32])

In [8]:
class ResidualAdd(nn.Module):
    def __init__(self, fn):
        super().__init__()
        self.fn = fn

    def forward(self, x, **kwargs):
        res = x
        x = self.fn(x, **kwargs)
        x += res
        return x

In [9]:
from einops import repeat

class ViT(nn.Module):
    def __init__(self, ch=1, img_size=28, patch_size=4, emb_dim=8,
                n_layers=2, out_dim=10, dropout=0., heads=2):
        super(ViT, self).__init__()

        # Attributes
        self.channels = ch
        self.height = img_size
        self.width = img_size
        self.patch_size = patch_size
        self.n_layers = n_layers

        # Patching
        self.patch_embedding = PatchEmbedding(in_channels=ch,
                                              patch_size=patch_size,
                                              emb_size=emb_dim)
        # Learnable params
        num_patches = (img_size // patch_size) ** 2
        self.pos_embedding = nn.Parameter(
            torch.randn(1, num_patches + 1, emb_dim))
        self.cls_token = nn.Parameter(torch.rand(1, 1, emb_dim))

        # Transformer Encoder
        self.layers = nn.ModuleList([])
        for _ in range(n_layers):
            transformer_block = nn.Sequential(
                ResidualAdd(PreNorm(emb_dim, Attention(emb_dim, n_heads = heads, dropout = dropout))),
                ResidualAdd(PreNorm(emb_dim, FeedForward(emb_dim, emb_dim, dropout = dropout))))
            self.layers.append(transformer_block)

        # Classification head
        self.head = nn.Sequential(nn.LayerNorm(emb_dim), nn.Linear(emb_dim, out_dim))
        
        #self.head_ln = nn.LayerNorm(emb_dim)
        #self.head = nn.Sequential(nn.Linear(int((1 + self.height/self.patch_size * self.width/self.patch_size) * emb_dim), out_dim))
        
        #self.head_ln = nn.LayerNorm(emb_dim)
        #self.head = nn.Sequential(nn.Linear(emb_dim, out_dim), nn.Softmax(dim=-1))
        
        
    def forward(self, img):
        # Get patch embedding vectors
        x = self.patch_embedding(img)
        b, n, _ = x.shape

        # Add cls token to inputs
        cls_tokens = repeat(self.cls_token, '1 1 d -> b 1 d', b = b)
        x = torch.cat([cls_tokens, x], dim=1)
        x += self.pos_embedding[:, :(n + 1)]

        # Transformer layers
        for i in range(self.n_layers):
            x = self.layers[i](x)

        # Output based on classification token
        return self.head(x[:, 0, :])

        #x = x.view(x.shape[0], int((1 + self.height/self.patch_size * self.width/self.patch_size) * x.shape[-1]))
        #out = self.head(x)
        #return out
        
        #out = self.head(x[:, 0])
        #return out

In [10]:
# Defining model and training options
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device: ", device, f"({torch.cuda.get_device_name(device)})" if torch.cuda.is_available() else "")
model = ViT(ch=1, img_size=28, patch_size=4, emb_dim=8, n_layers=2, out_dim=10, dropout=0., heads=2).to(device)
optimizer = optim.AdamW(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
n_epochs = 10

# torch
inputs, labels = next(iter(train_dataloader))
inputs, labels = inputs.to(device), labels.to(device)
outputs = model(inputs)
patches = model.patch_embedding(inputs)
print("Shape of example: {}".format(inputs.shape))
print("Shape of Patches: {}".format(patches.shape))
print("Shape of output: {}".format(outputs.shape))

Using device:  cuda (NVIDIA GeForce RTX 3070 Ti)
Shape of example: torch.Size([128, 1, 28, 28])
Shape of Patches: torch.Size([128, 49, 8])
Shape of output: torch.Size([128, 10])


In [11]:
# Main training routine
def train(epoch):
    model.train()
    for step, (inputs, labels) in enumerate(train_dataloader):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        # Store results
        if step % 8 == 0:
            print('Train Epoch: {} [{}/{}]\tLoss: {:.6f}'.format(
                epoch, step * len(inputs), len(train_dataloader.dataset), loss.item()))

In [12]:
# Run on test data
def test():
    epoch_losses = []
    test_loss = 0
    correct = 0
    # Something was strange when using this?
    # model.eval()
    for step, (inputs, labels) in enumerate(test_dataloader):
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        epoch_losses.append(loss.item())
        pred = outputs.data.max(1, keepdim=True)[1]
        correct += pred.eq(labels.data.view_as(pred)).sum()
    test_loss = np.mean(epoch_losses)
    print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_dataloader.dataset),
        100. * correct / len(test_dataloader.dataset)))

In [13]:
# Get initial performance
#test()
# Train for three epochs
for epoch in range(1, n_epochs + 1):
    train(epoch)
    test()

Train Epoch: 1 [0/60000]	Loss: 2.527013
Train Epoch: 1 [1024/60000]	Loss: 2.433336
Train Epoch: 1 [2048/60000]	Loss: 2.416459
Train Epoch: 1 [3072/60000]	Loss: 2.371042
Train Epoch: 1 [4096/60000]	Loss: 2.337822
Train Epoch: 1 [5120/60000]	Loss: 2.316396
Train Epoch: 1 [6144/60000]	Loss: 2.320098
Train Epoch: 1 [7168/60000]	Loss: 2.298880
Train Epoch: 1 [8192/60000]	Loss: 2.306011
Train Epoch: 1 [9216/60000]	Loss: 2.264112
Train Epoch: 1 [10240/60000]	Loss: 2.272666
Train Epoch: 1 [11264/60000]	Loss: 2.257117
Train Epoch: 1 [12288/60000]	Loss: 2.218036
Train Epoch: 1 [13312/60000]	Loss: 2.214975
Train Epoch: 1 [14336/60000]	Loss: 2.161872
Train Epoch: 1 [15360/60000]	Loss: 2.066006
Train Epoch: 1 [16384/60000]	Loss: 2.037601
Train Epoch: 1 [17408/60000]	Loss: 2.061960
Train Epoch: 1 [18432/60000]	Loss: 2.036791
Train Epoch: 1 [19456/60000]	Loss: 1.973611
Train Epoch: 1 [20480/60000]	Loss: 1.958320
Train Epoch: 1 [21504/60000]	Loss: 1.926242
Train Epoch: 1 [22528/60000]	Loss: 1.861042
T